# StyleGAN 3

## Preprocess

In [ ]:
!pip install einops ninja gdown click scipy tensorboard

In [ ]:
import os

# Define the static dataset information
DATASET = {
    "name": "project_name",
    "path": "path/to/dataset",
    "res": "512x512" # What it will preprocess to
}

# Function to calculate kimg based on epochs
def calculate_kimg(num_images, epochs):
    return (num_images * epochs) // 1000

# Define the number of images to test
NUM_IMAGES = [500, 1000, 2500]

# Define the fixed number of epochs
EPOCHS = 100

training_configs = []
for num_images in NUM_IMAGES:
    kimg = calculate_kimg(num_images, EPOCHS)
    training_configs.append({
        "num_images": num_images,
        "kimg": kimg
    })

# Print the configurations for verification
print("Training Configurations:")
for config in training_configs:
    print(f"Number of Images: Epochs: {EPOCHS}, {config['num_images']}, kimg: {config['kimg']}")


In [ ]:
import os

# Define the path to the directory
dir_path = f"Results/{DATASET['name']}/model"

os.makedirs(dir_path, exist_ok=True)

# Check if the directory is empty
if not os.listdir(dir_path):
    %cd Results/{DATASET['name']}/model/

    !git clone https://github.com/NVlabs/stylegan3

# Prepare the path using f-string
stylegan3_path = f"Results/{DATASET['name']}/model/stylegan3"

# Use the formatted path in the %cd command
%cd {stylegan3_path}

In [ ]:
from PIL import Image
import os
import random

# Parameters
DATASET_PATH = "path/to/dataset"
OUTPUT_DIR = "Results/dataset_name/preprocessed"

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_image_paths(directory_path):
    image_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpeg', '.jpg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

def sample_and_convert_images(image_paths, num_samples, output_dir):
    sampled_paths = random.sample(image_paths, min(num_samples, len(image_paths)))
    for image_path in sampled_paths:
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, filename)
        image = Image.open(image_path)
        image = image.convert("RGB")
        image.save(output_path)
        print(f"Converted {filename} to RGB and saved to {output_path}")

# Get all image paths from the dataset
image_paths = get_image_paths(DATASET_PATH)

# Specify the number of images to sample
num_samples = 2500  # Adjust this number as needed

# Sample and convert images
sample_and_convert_images(image_paths, num_samples, OUTPUT_DIR)


## 500 Images 

### Training

In [ ]:
config = training_configs[0]

In [ ]:
import os
import shutil

# Directory containing all preprocessed images
PREPROCESSED_DIR = f"Results/{DATASET['name']}/preprocessed"
# Temporary directory to store the first 500 images
TEMP_DIR = f"Results/{DATASET['name']}/{config['num_images']}_preprocessed"

# Ensure the temporary directory exists
os.makedirs(TEMP_DIR, exist_ok=True)

# Get all image filenames in the preprocessed directory
all_images = [f for f in os.listdir(PREPROCESSED_DIR) if f.lower().endswith(('.png', '.jpeg', '.jpg'))]

# Sort and select the first X images
divided_images = sorted(all_images)[:config['num_images']]

# Copy the first 500 images to the temporary directory
for image in divided_images:
    src_path = os.path.join(PREPROCESSED_DIR, image)
    dest_path = os.path.join(TEMP_DIR, image)
    shutil.copy(src_path, dest_path)
    print(f"Copied {image} to {TEMP_DIR}")

In [ ]:
dest_file = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"

!python path/to/dataset_tool.py \
    --source='{TEMP_DIR}' \
    --dest='{dest_file}' \
    --resolution={DATASET['res']}

# Delete the temporary directory after processing
shutil.rmtree(TEMP_DIR)
print(f"Temporary directory {TEMP_DIR} deleted.")

In [ ]:
checkpoints_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints"

# Create directories if they don't exist
os.makedirs(checkpoints_dir, exist_ok=True)

training_data = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"

# Start training with adjusted parameters
!python path/to/train.py \
  --outdir='{checkpoints_dir}' --cfg=stylegan3-t \
  --data='{training_data}' \
  --snap=10 --metrics=fid50k_full --gpus=3 --batch=96 --batch-gpu=8 --gamma=8 --mirror=1 --kimg={config["kimg"]}

### Generating Images

In [ ]:
generated_images_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/generated_images"

# Create directories if they don't exist
os.makedirs(generated_images_dir, exist_ok=True)

gen_network = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints/00001-stylegan3-t-preprocessed-512x512-gpus3-batch96-gamma8/network-snapshot-0000{config['kimg']}.pkl"

# Generate images using the trained model
!python path/to/gen_images.py \
  --outdir='{generated_images_dir}' \
  --trunc=0.7 --seeds=0-24 \
  --network='{gen_network}'

## 1000 Images

### Training

In [ ]:
config = training_configs[1]

In [ ]:
import os
import shutil

# Directory containing all preprocessed images
PREPROCESSED_DIR = f"Results/{DATASET['name']}/preprocessed"
# Temporary directory to store the first 500 images
TEMP_DIR = f"Results/{DATASET['name']}/{config['num_images']}_preprocessed"

# Ensure the temporary directory exists
os.makedirs(TEMP_DIR, exist_ok=True)

# Get all image filenames in the preprocessed directory
all_images = [f for f in os.listdir(PREPROCESSED_DIR) if f.lower().endswith(('.png', '.jpeg', '.jpg'))]

# Sort and select the first X images
divided_images = sorted(all_images)[:config['num_images']]

# Copy the first X images to the temporary directory
for image in divided_images:
    src_path = os.path.join(PREPROCESSED_DIR, image)
    dest_path = os.path.join(TEMP_DIR, image)
    shutil.copy(src_path, dest_path)
    print(f"Copied {image} to {TEMP_DIR}")

In [ ]:
dest_file = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"

!python path/to/dataset_tool.py \
    --source='{TEMP_DIR}' \
    --dest='{dest_file}' \
    --resolution={DATASET['res']}

# Delete the temporary directory after processing
shutil.rmtree(TEMP_DIR)
print(f"Temporary directory {TEMP_DIR} deleted.")

In [ ]:
checkpoints_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints"

# Create directories if they don't exist
os.makedirs(checkpoints_dir, exist_ok=True)

training_data = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"

# Start training with adjusted parameters
!python path/to/train.py \
  --outdir='{checkpoints_dir}' --cfg=stylegan3-t \
  --data='{training_data}' \
  --snap=10 --metrics=fid50k_full --gpus=3 --batch=24 --batch-gpu=8 --gamma=8 --mirror=1 --kimg={config["kimg"]}

### Generating Images

In [ ]:
generated_images_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/generated_images"

# Create directories if they don't exist
os.makedirs(generated_images_dir, exist_ok=True)

gen_network = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints/00002-stylegan3-t-preprocessed-512x512-gpus3-batch24-gamma8/network-snapshot-000{config['kimg']}.pkl"

# Generate images using the trained model
!python path/to/gen_images.py \
  --outdir='{generated_images_dir}' \
  --trunc=1 --seeds=0-24 \
  --network='{gen_network}'

## 2500 Images

### Training

In [ ]:
config = training_configs[2]

In [ ]:
dest_file = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"
FULL_PREPROCESSED = f"Results/{DATASET['name']}/preprocessed"

!python path/to/dataset_tool.py \
    --source='{FULL_PREPROCESSED}' \
    --dest='{dest_file}' \
    --resolution={DATASET['res']}

In [ ]:
checkpoints_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints"

# Create directories if they don't exist
os.makedirs(checkpoints_dir, exist_ok=True)

training_data = f"Results/{DATASET['name']}/{config['num_images']}_images/preprocessed-{DATASET['res']}.zip"

# Start training with adjusted parameters
!python path/to/train.py \
  --outdir='{checkpoints_dir}' --cfg=stylegan3-t \
  --data='{training_data}' \
  --snap=10 --metrics=fid50k_full --gpus=3 --batch=24 --batch-gpu=8 --gamma=8 --mirror=1 --kimg={config["kimg"]} 


### Generating Images

In [ ]:
generated_images_dir = f"Results/{DATASET['name']}/{config['num_images']}_images/generated_images"

# Create directories if they don't exist
os.makedirs(generated_images_dir, exist_ok=True)

# MAKE SURE TO CHANGE MANUALLY TO CORRECT PATH
gen_network = f"Results/{DATASET['name']}/{config['num_images']}_images/checkpoints/00004-stylegan3-t-preprocessed-512x512-gpus3-batch24-gamma8/network-snapshot-000{config['kimg']}.pkl"

# Generate images using the trained model
!python path/to/gen_images.py \
  --outdir='{generated_images_dir}' \
  --trunc=0.5 --seeds=0-24 \
  --network='{gen_network}'